# Sistema de Recomendación

In [5]:
# imports
import pandas as pd
from funciones_scraping import get_languages

df = pd.read_csv('../output/df-limpio-iter3.csv', index_col=0)
df.shape

(11428, 55)

In [6]:
languages = get_languages()

In [7]:
def normalize_columns(data, columns): 
    for col in columns: 
        if data[col].max() > 0: 
            data[col] = data[col]/data[col].max()
    return data

df_norm = normalize_columns(df, languages)
df_norm.reset_index(drop=True, inplace=True)


In [8]:
df_norm.describe()

,allies,bf (beta),c,c#,c++,clojure,coffeescript,collections,comments,crystal,...,scala (beta),shell,solidity (beta),sql,swift,total completed kata,total languages trained,translations,typescript,time
count,11428.000000,11428.0,11428.000000,11428.0,11428.0,11428.000000,11428.000000,11428.000000,11428.000000,11428.000000,...,11428.0,11428.000000,11428.0,11428.000000,11428.000000,11428.000000,11428.000000,11428.000000,11428.000000,11428.000000
mean,93.289902,0.0,0.001295,0.0,0.0,0.001161,0.000957,0.143857,13.353255,0.000312,...,0.0,0.000551,0.0,0.009283,0.000642,102.076041,2.412496,1.488799,0.001119,496.643420
std,232.583439,0.0,0.019172,0.0,0.0,0.017324,0.014934,0.843151,173.395939,0.011377,...,0.0,0.012697,0.0,0.056273,0.015932,289.016703,2.597626,39.647146,0.020352,461.830188
min,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,9.000000,1.000000,0.000000,0.000000,91.000000
50%,15.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,28.000000,2.000000,0.000000,0.000000,395.000000
75%,72.250000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,76.000000,3.000000,0.000000,0.000000,791.000000
max,10610.000000,0.0,1.000000,0.0,0.0,1.000000,1.000000,29.000000,8962.000000,1.000000,...,0.0,1.000000,0.0,1.000000,1.000000,7811.000000,38.000000,2927.000000,1.000000,2252.000000


In [9]:
def ask_languages(languages=languages): 
    while True: 
        try: 
            res = input('Introduce los tags(lenguajes) separados por coma:').lower().split(',')
            res = [r.strip() for r in res]
            break
        except: 
            print('Fallo en la captura de requisitos. Inténtalo de nuevo')
    return res


In [10]:
# ask_languages()
skills = ask_languages()

Introduce los tags(lenguajes) separados por coma:python, java, c, javascript


In [11]:
def get_ask_vector(skills, languages): 
    res = sorted([(l, 1) if l in skills else (l, 0) for l in languages])
    return [v for k, v in res]
# get_ask_vector(['python', 'c'], get_languages())       

def distacia_euclidea(v1, v2): 
    return sum([(x1-x2)**2 for x1, x2 in zip(v1, v2)])**0.5
# distacia_euclidea([3, 4], [0, 0])

def get_languages_vector(row, languages):
    # return [(k, v) for k, v in row.items() if k in languages]
    res = sorted([(k, v) for k, v in row.items() if k in languages])
    return [v for k, v in res]
# get_languages_vector(df_norm.loc[4, :], get_languages())

v1 = get_ask_vector(skills, languages)

In [12]:
distacia_euclidea(v1, get_languages_vector(df_norm.loc[55, :], languages))

1.8228939782543003

In [13]:
# df_norm['distancia'] = df_norm.apply(lambda x: distacia_euclidea(v1, get_languages_vector(x, languages)))

In [14]:
distancias = []
for i in df_norm.index.values: 
    distancias.append( distacia_euclidea(v1, get_languages_vector(df_norm.loc[i, :], languages)) )
len(distancias)    

11428

In [15]:
df_norm['distancia'] = distancias

In [16]:
df_sort = df_norm.sort_values(by=['distancia'])
df_sort.head()

,allies,bf (beta),c,c#,c++,clojure,coffeescript,collections,comments,crystal,...,swift,total completed kata,total languages trained,translations,typescript,username,github,linkedin,time,distancia
3330,19.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,11.0,7358.0,0.0,...,0.000000,3297.0,9.0,190.0,0.000000,Blind4Basics,https://github.com/Blind4Basics,NaN,821,1.427804
1442,6.0,0.0,0.225595,0.0,0.0,0.117769,0.000000,3.0,134.0,0.0,...,0.000000,1750.0,17.0,28.0,0.036399,monadius,https://github.com/monadius,https://www.linkedin.com/in/alexey-solovyev-52...,334,1.511648
253,229.0,0.0,0.086820,0.0,0.0,0.000000,0.023428,4.0,301.0,0.0,...,0.000000,4009.0,12.0,47.0,0.000000,Bubbler,https://github.com/Bubbler-4,NaN,427,1.552670
5188,0.0,0.0,1.000000,0.0,0.0,0.014463,0.000290,0.0,208.0,0.0,...,0.001928,554.0,18.0,89.0,0.000000,nomennescio,https://github.com/nomennescio,NaN,700,1.593529
4118,0.0,0.0,0.959119,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,93.0,6.0,0.0,0.000000,Skoopy,https://github.com/Skoopy33,NaN,61,1.599482
